In [1]:
import logging
logging.basicConfig(filename='logger.log', level=logging.INFO)
import pandas as pd

In [2]:
# 读取
before_anno_dir='..\data\seg_anno.csv'
df=pd.read_csv(before_anno_dir,sep='\t',index_col=0)
df.head()

,src_idx,keywords,text,mention,entity_list,annotation
0,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,日本/GPE,"{1: '日本（东亚国家）', 2: '日本（中国旅游出版社图书）', 3: '日本（山名）...",0
1,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,韩国/GPE,"{1: '韩国（周朝诸侯国）', 2: '韩国（大宝石出版社图书）', 3: '韩国（社会科...",0
2,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,王晶/PERSON,"{1: '王晶（东北师范大学政法学院）', 2: '王晶（山东艺术学院音乐学院教师）', 3...",0
3,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,北京/GPE,"{1: '北京（歌手郝云专辑）', 2: '北京（北宋四京之一北京大名府）', 3: '北京...",0
4,0,信息泄露,“当前居民住房杠杆率较高，需要密切关注过度借贷导致的居民杠杆率继续上升的问题。此前还有部分消...,何飞/PERSON,"{1: '何飞（塘红乡党委统委、副乡长）', 2: '何飞（四川省新津县兴义镇党委委员）',...",0


In [5]:
kw_cnts=df['keywords'].value_counts()# 统计关键词文档数量
print(kw_cnts)
kw_cnts.to_excel('../data/eda/kw_cnts.xlsx',sheet_name='关键词分布')

战略合作      11240
违规担保       2303
禁入         2263
占用资金       2143
市场禁入       2143
非法集资       1669
重大遗漏       1420
误导性陈述      1381
责令整改       1274
公开谴责       1270
信息泄露       1119
监管关注函      1119
隐瞒关联关系      525
强制摘牌        358
虚报注册资本      348
秋后算账        290
票据违法         84
股东虚假出资       41
Name: keywords, dtype: int64


In [12]:
# 合并后缀为2的关键词
df['keywords']=df['keywords'].apply(lambda x: x[:-1] if x.endswith('2') else x)

In [13]:
df['keywords'].value_counts()

信息泄露    6
Name: keywords, dtype: int64

In [ ]:
# 关键词分布图
import seaborn as sns
sns.set_style('whitegrid',{'font.sans-serif':['simhei','Arial']})
sns.countplot(y=df['keywords'])

In [ ]:
df['len_text'].describe() # 文本长度分布

In [14]:
# df.to_csv('fin_articles.csv',encoding='utf-8',index=0)
df.to_csv('test_fin_articles.csv',encoding='utf-8',index=0)

In [15]:
df2=pd.read_csv('test_fin_articles.csv',encoding='utf8')
df2.head()

,src_web,keywords,text,entities,len_text
0,人民网产经,信息泄露,消费火正旺 金融添把柴\n年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增...,"[{'text': '韩国', 'type': 'GPE', 'start_char': 8...",3241
1,人民网产经,信息泄露,又一老人被骗280万元！这种诈骗来势凶猛 务必学会辨认\n广州市公安局9月2日发出紧急通报：...,"[{'text': '280万元', 'type': 'MONEY', 'start_cha...",1195
2,人民网产经,信息泄露,"月花两三千，过得像月入三五万？""租一族""崛起！但真的划算吗？\n原标题：月花两三千，过得像月...","[{'text': '三千', 'type': 'CARDINAL', 'start_cha...",2501
3,人民网产经,信息泄露,2020年我国数据总量全球占比将达20% 信息基础设施保护是大数据安全关键\n原标题：202...,"[{'text': '2020年', 'type': 'DATE', 'start_char...",1926
4,人民网产经,信息泄露,1秒钟能下载一部高清电影 5G来了！你准备好了吗\n原标题：5G来了！你准备好了吗\n5月1...,"[{'text': '1秒', 'type': 'TIME', 'start_char': ...",2030


In [ ]:
# 抽取实体